# import

In [47]:
import os
import numpy
import json
import codecs
import ast
import pandas as pd
import gc
import hashlib
import itertools
import json
import os
import re
import subprocess
import time
import collections
import six
import torch
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request

from gensim.models.word2vec import Word2Vec
from glob import glob
from multiprocess import Pool
from pytorch_pretrained_bert import BertTokenizer

from tqdm import tqdm
from konlpy.tag import Komoran
from pandas import json_normalize

# create csv

In [ ]:
full_json = json.load(codecs.open('dataset/original_json/train.json', 'r', 'utf-8-sig'))
test_json = json.load(codecs.open('dataset/original_json/test.json', 'r', 'utf-8-sig'))
full_json

In [ ]:
full_df = json_normalize(full_json)
test_df = json_normalize(test_json)
full_df.head()

In [ ]:
full_df['extractive_sents'] = full_df.apply(lambda row: list(np.array(row['article_original'])[row['extractive']]) , axis=1)
full_df.head()

In [ ]:
train_df = full_df.sample(frac=0.95,random_state=42) #random state is a seed value
valid_df = full_df.drop(train_df.index)
train_df.reset_index(inplace=True, drop=True)
valid_df.reset_index(inplace=True, drop=True)

In [ ]:
train_df.head()

In [ ]:
valid_df.head()

# 형태소 분석

In [ ]:
komoran = Komoran()

In [ ]:
def converter_morp(text):
    conv_text = []
    for i in range(len(text)):
        conv_text.append(komoran.pos(text[i],join=True))
    return conv_text

In [ ]:
tqdm.pandas()

train_df['article_morp'] = train_df['article_original'].progress_apply(converter_morp)
train_df.head()

In [ ]:
valid_df['article_morp'] = valid_df['article_original'].progress_apply(converter_morp)
valid_df.head()

In [ ]:
test_df['article_morp'] = test_df['article_original'].progress_apply(converter_morp)
test_df.head()

In [ ]:
train_df.to_csv('dataset/csv/train.csv',index=False)
valid_df.to_csv('dataset/csv/valid.csv',index=False)
test_df.to_csv('dataset/csv/test.csv',index=False)

In [ ]:
temp_df = train_df.loc[0:3]
temp_df.head()

# load df && df to json

In [31]:
train_df = pd.read_csv('dataset/csv/train.csv')
valid_df = pd.read_csv('dataset/csv/valid.csv')
test_df = pd.read_csv('dataset/csv/test.csv')

In [32]:
train_df['extractive'] = train_df['extractive'].apply(ast.literal_eval)
train_df['article_original'] = train_df['article_original'].apply(ast.literal_eval)
train_df['extractive_sents'] = train_df['extractive_sents'].apply(ast.literal_eval)
train_df['article_morp'] = train_df['article_morp'].apply(ast.literal_eval)

In [33]:
valid_df['extractive'] = valid_df['extractive'].apply(ast.literal_eval)
valid_df['article_original'] = valid_df['article_original'].apply(ast.literal_eval)
valid_df['extractive_sents'] = valid_df['extractive_sents'].apply(ast.literal_eval)
valid_df['article_morp'] = valid_df['article_morp'].apply(ast.literal_eval)

In [34]:
test_df['article_original'] = test_df['article_original'].apply(ast.literal_eval)
test_df['article_morp'] = test_df['article_morp'].apply(ast.literal_eval)

In [6]:
train_df.head()

,id,extractive,article_original,extractive_sents,article_morp
0,191521,"[9, 7, 19]","[[1] 의장이 동일·유사하다고 하려면, 우선 의장이 표현된 물품이 동일·유사하여야...",[[3] 의장의 유사 여부는 이를 구성하는 각 요소를 분리하여 개별적으로 대비할 것...,"[[[/SS, 1/SN, ]/SS, 의장/NNG, 이/JKS, 동일/NNG, ·/S..."
1,102453,"[0, 3, 4]","[가. 자동차운수사업법 제31조 제5호 소정의 ""중대한 교통사고""에 해당하는지의 여...","[가. 자동차운수사업법 제31조 제5호 소정의 ""중대한 교통사고""에 해당하는지의 여...","[[가/VV, 아/EF, ./SF, 자동차/NNP, 운수/NNP, 사업/NNP, 법..."
2,83026,"[0, 1, 6]","[[1] 형사재판에 있어서 공소가 제기된 범죄사실에 대한 입증책임은 검사에 있고,,...","[[1] 형사재판에 있어서 공소가 제기된 범죄사실에 대한 입증책임은 검사에 있고,,...","[[[/SS, 1/SN, ]/SS, 형사/NNP, 재판/NNP, 에/JKB, 있/V..."
3,156607,"[0, 2, 3]",[어음행위의 대리에 있어서는 형식적 요건으로서 본인을 위한다는 대리의 문구를 기재하...,[어음행위의 대리에 있어서는 형식적 요건으로서 본인을 위한다는 대리의 문구를 기재하...,"[[어음/NNP, 행위/NNP, 의/JKG, 대리/NNP, 에/JKB, 있/VV, ..."
4,196021,"[0, 3, 4]",[구 병역법(2009. 6. 9. 법률 제9754호로 개정되기 전의 것) 제70조 ...,[구 병역법(2009. 6. 9. 법률 제9754호로 개정되기 전의 것) 제70조 ...,"[[구/NNG, 병역법/NNP, (/SS, 2009/SN, ./SF, 6/SN, ...."


In [11]:
def csv_to_json(df,mode):
    list_dic = []
    for idx, row in df.iterrows():
        mydict = {}
        mydict['src'] = row['article_morp']
        
        if mode is not 'test':
            mydict['tgt'] = [a for i,a in enumerate(mydict['src']) if i in row['extractive']]
        
        list_dic.append(mydict)
    temp = []
    for i,a in enumerate(list_dic):
        if (i+1)%6!=0:
            temp.append(a)
        else:
            filename = 'korean.'+mode+'.'+str(i//6)+'.json'
            with open('dataset/converter_json/'+mode+'/'+filename, "w", encoding='utf8') as json_file:
                json.dump(temp, json_file, ensure_ascii=False)
            temp = []

In [12]:
csv_to_json(train_df,'train')
csv_to_json(valid_df,'valid')
csv_to_json(test_df,'test')

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

# word2vec

In [35]:
train_df = pd.read_csv('dataset/csv/train.csv')
train_df['extractive'] = train_df['extractive'].apply(ast.literal_eval)
train_df['article_original'] = train_df['article_original'].apply(ast.literal_eval)
train_df['extractive_sents'] = train_df['extractive_sents'].apply(ast.literal_eval)
train_df['article_morp'] = train_df['article_morp'].apply(ast.literal_eval)

,id,extractive,article_original,extractive_sents,article_morp
0,191521,"[9, 7, 19]","[[1] 의장이 동일·유사하다고 하려면, 우선 의장이 표현된 물품이 동일·유사하여야...",[[3] 의장의 유사 여부는 이를 구성하는 각 요소를 분리하여 개별적으로 대비할 것...,"[[[/SS, 1/SN, ]/SS, 의장/NNG, 이/JKS, 동일/NNG, ·/S..."
1,102453,"[0, 3, 4]","[가. 자동차운수사업법 제31조 제5호 소정의 ""중대한 교통사고""에 해당하는지의 여...","[가. 자동차운수사업법 제31조 제5호 소정의 ""중대한 교통사고""에 해당하는지의 여...","[[가/VV, 아/EF, ./SF, 자동차/NNP, 운수/NNP, 사업/NNP, 법..."
2,83026,"[0, 1, 6]","[[1] 형사재판에 있어서 공소가 제기된 범죄사실에 대한 입증책임은 검사에 있고,,...","[[1] 형사재판에 있어서 공소가 제기된 범죄사실에 대한 입증책임은 검사에 있고,,...","[[[/SS, 1/SN, ]/SS, 형사/NNP, 재판/NNP, 에/JKB, 있/V..."
3,156607,"[0, 2, 3]",[어음행위의 대리에 있어서는 형식적 요건으로서 본인을 위한다는 대리의 문구를 기재하...,[어음행위의 대리에 있어서는 형식적 요건으로서 본인을 위한다는 대리의 문구를 기재하...,"[[어음/NNP, 행위/NNP, 의/JKG, 대리/NNP, 에/JKB, 있/VV, ..."
4,196021,"[0, 3, 4]",[구 병역법(2009. 6. 9. 법률 제9754호로 개정되기 전의 것) 제70조 ...,[구 병역법(2009. 6. 9. 법률 제9754호로 개정되기 전의 것) 제70조 ...,"[[구/NNG, 병역법/NNP, (/SS, 2009/SN, ./SF, 6/SN, ...."


In [40]:
tokenized_data = sum(tokenized_data,[])
tokenized_data[:5]

[['[/SS',
  '1/SN',
  ']/SS',
  '의장/NNG',
  '이/JKS',
  '동일/NNG',
  '·/SP',
  '유사/XR',
  '하/XSA',
  '다고/EC',
  '하/VV',
  '려면/EC'],
 ['우선/MAG',
  '의장/NNG',
  '이/JKS',
  '표현/NNG',
  '되/XSV',
  'ㄴ/ETM',
  '물품/NNG',
  '이/JKS',
  '동일/NNG',
  '·/SP',
  '유사/XR',
  '하/XSA',
  '아야/EC',
  '하/VV',
  'ㄹ/ETM',
  '것/NNB',
  '이/VCP',
  'ㄴ바/EC',
  ',/SP'],
 ['물품/NNG',
  '의/JKG',
  '동일/NNG',
  '·/SP',
  '유사/NNP',
  '성/XSN',
  '여부/NNG',
  '는/JX',
  '물품/NNG',
  '의/JKG',
  '용도/NNG',
  ',/SP'],
 ['기능/NNG',
  '등/NNB',
  '에/JKB',
  '비추/VV',
  '어/EC',
  '거래/NNG',
  '통념/NNG',
  '상/XSN',
  '동일/NNG',
  '·/SP',
  '유사/NNP',
  '한/NNP',
  '물품/NNG',
  '으로/JKB',
  '인정/NNG',
  '하/XSV',
  'ㄹ/ETM',
  '수/NNB',
  '있/VV',
  '는지/EC',
  '여부/NNG',
  '에/JKB',
  '따르/VV',
  '아/EC',
  '결정/NNG',
  '하/XSV',
  '아야/EC',
  '하/VV',
  'ㄹ/ETM',
  '것/NNB',
  '이/VCP',
  '고/EC',
  ',/SP'],
 ['의장/NNP',
  '법/NNG',
  '시행/NNG',
  '규칙/NNP',
  '제/XPN',
  '9/SN',
  '조/NR',
  '제/XPN',
  '1/SN',
  '항/NNG',
  '소정/NNP',
  '의/JKG',
  '물품/NNG',
  '구분/NNG'

In [43]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, 
                 #size = 100,
                 window = 5, 
                 min_count = 5, 
                 workers = 4, 
                 sg = 0)

# preprocessing

In [18]:
def convert_to_unicode(text):
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")



In [19]:
class Tokenizer(object):
    def __init__(self, vocab_file_path):
        self.vocab_file_path = vocab_file_path
        """Loads a vocabulary file into a dictionary."""
        vocab = collections.OrderedDict()
        index = 0
        with open(self.vocab_file_path, "r", encoding='utf-8') as reader:

            while True:
                token = convert_to_unicode(reader.readline())
                if not token:
                    break

          ### joonho.lim @ 2019-03-15
                if token.find('n_iters=') == 0 or token.find('max_length=') == 0 :

                    continue
                token = token.split('\t')[0].strip('_')

                token = token.strip()
                vocab[token] = index
                index += 1
        self.vocab = vocab
    def convert_tokens_to_ids(self, tokens):
        """Converts a sequence of tokens into ids using the vocab."""
        ids = []
        for token in tokens:
            try:
                ids.append(self.vocab[token])
            except:
                ids.append(1)
        if len(ids) > 10000:
            raise ValueError(
                "Token indices sequence length is longer than the specified maximum "
                " sequence length for this BERT model ({} > {}). Running this"
                " sequence through BERT will result in indexing errors".format(len(ids), 10000)
            )
        return ids

In [20]:
class BertData():
    def __init__(self,vocab_file_path):
        self.tokenizer = Tokenizer(vocab_file_path)
        self.sep_vid = self.tokenizer.vocab['[SEP]']
        self.cls_vid = self.tokenizer.vocab['[CLS]']
        self.pad_vid = self.tokenizer.vocab['[PAD]']
        
        self.min_src_ntokens = 5
        self.max_src_ntokens = 200
        self.max_nsents = 100
        self.min_nsents = 3
        
    def preprocess(self, src, tgt, oracle_ids):

        if (len(src) == 0):
            return None

        original_src_txt = [' '.join(s) for s in src]

        labels = [0] * len(src)
        for l in oracle_ids:
            labels[l] = 1

        idxs = [i for i, s in enumerate(src) if (len(s) > self.min_src_ntokens)]

        src = [src[i][:self.max_src_ntokens] for i in idxs]
        labels = [labels[i] for i in idxs]
        src = src[:self.max_nsents]
        labels = labels[:self.max_nsents]

        if (len(src) < self.min_nsents):
            return None
        if (len(labels) == 0):
            return None

        src_txt = [' '.join(sent) for sent in src]
        # text = [' '.join(ex['src_txt'][i].split()[:self.max_src_ntokens]) for i in idxs]
        # text = [_clean(t) for t in text]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = text.split(' ')
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = labels[:len(cls_ids)]

        tgt_txt = '<q>'.join([' '.join(tt) for tt in tgt])
        src_txt = [original_src_txt[i] for i in idxs]
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt

In [24]:
def _format_to_bert(params,oracle_mode):
    json_file, save_file = params
    if (os.path.exists(save_file)):
        #logger.info('Ignore %s' % save_file)
        return

    bert = BertData(vocab_file_path)

    #logger.info('Processing %s' % json_file)
    jobs = json.load(open(json_file))
    datasets = []
    for d in jobs:
        source, tgt = d['src'], d['tgt']
        if (oracle_mode == 'greedy'):
            oracle_ids = greedy_selection(source, tgt, 3)
        elif (oracle_mode == 'combination'):
            oracle_ids = combination_selection(source, tgt, 3)
        b_data = bert.preprocess(source, tgt, oracle_ids)
        if (b_data is None):
            continue
        indexed_tokens, labels, segments_ids, cls_ids, src_txt, tgt_txt = b_data
        b_data_dict = {"src": indexed_tokens, "labels": labels, "segs": segments_ids, 'clss': cls_ids,
                       'src_txt': src_txt, "tgt_txt": tgt_txt}
        datasets.append(b_data_dict)
    #logger.info('Saving to %s' % save_file)
    torch.save(datasets, save_file)
    datasets = []
    gc.collect()

In [25]:
def format_to_bert(json_path, dataset_type, save_path, oracle_mode):
    a_lst = []
    for json_f in tqdm(glob(os.path.join(json_path, '*' + dataset_type + '.*.json'))):
        real_name = json_f.split('/')[-1]
        print((json_f, os.path.join(save_path, real_name.replace('json', 'bert.pt'))))
        break
        #_format_to_bert((json_f, os.path.join(args.save_path, real_name.replace('json', 'bert.pt'))),oracle_mode)
        #a_lst.append((json_f, os.path.join(args.save_path, real_name.replace('json', 'bert.pt'))))
    print(a_lst)

In [26]:
train_json_path = 'dataset/converter_json/train/'
valid_json_path = 'dataset/converter_json/valid/'
oracle_mode = 'greedy' # or 'combination'

In [27]:
format_to_bert(train_json_path,'train','dataset/bert/train/',oracle_mode)

('dataset/converter_json/train/korean.train.1303.json', 'dataset/bert/train/korean.train.1303.bert.pt')
[]
